# Lab 2 - Parte 1 - Reposição

## Word2Vec

Esta atividade foi realizada com o propósito de aprender sobre e construir um modelo Word2Vec. Este algoritmo consiste em transformar palavras em vetores, de forma a dispô-las em um espaço vetorial e tornando possível realização de operações matemáticas. Neste exercício, o foco será na distância entre tokens vetorizados. 

## Início

Inicialmente, faz-se os imports das bibliotecas que serão utilizadas e define-se a __função de padronização de texto__, que serve para remoção de __caracteres especiais__ e para deixar todas as palavras em __letras minúsculas__:

In [83]:
import csv
from unidecode import unidecode
import warnings
import gensim
warnings.filterwarnings('ignore')

def standardize_text(text):
    return unidecode(''.join(e for e in text if (e.isalnum() or e ==' ')).lower())


Feito isso, deseja-se agora __carregar os dados__ de estudo, que neste caso trata-se de uma coleção de notícias do portal Estadão. Neste processo, __três coisas__ devem ser feitas:
1. Concatenar título, sub-título e corpo das notícias;
2. Usar a função de padronização no resultado da concatenação;
3. Criar uma lista de tokens(palavras) usando o split

In [87]:
documents = []

with open('estadao_noticias_eleicao.csv', 'rt', encoding='utf8') as csvfile:
    estadao = csv.reader(csvfile)
    for index, article in enumerate(estadao):
        if index == 0:
            continue
            
        article_content = article[1] + " " + article[2] + " " + article[3]
        documents.append(standardize_text(article_content).split(' '))

É possível, agora, observar uma parte do resultado de carregamento para o primeiro documento da coleção:

In [88]:
documents[0][:10]

['pt',
 'espera',
 '30',
 'mil',
 'pessoas',
 'em',
 'festa',
 'na',
 'esplanada',
 'objetivo']

## Treinamento de modelo

Com os dados necessários em mãos, se torna simples __treinar o modelo__. Deve ser passado para a biblioteca Gensim, uma __lista de listas de palavras__ (definida na seção anterior), __gerar o modelo__, e __treiná-lo__.

In [33]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

(27508697, 36779450)

## Resultados 

O treinamento do modelo pode demorar um pouco. Por baixo, o Gensim constroi uma rede neural, da qual deseja-se extrair os pesos gerados, que representam os vetores das palavras. 
O Gensim prover, também, um conjunto de funções que tornam trivial a __obtenção de respostas para perguntas que podem ser muito relevantes__, como: 
1. __Uma palavra está relacionada com uma outra?__
2. __Dado um conjunto de palavras, existe alguma que não faz sentido estar naquele conjunto?__
3. __Dada uma palavra, quais são as n palavras mais parecidas com esta?__

### Similaridade entre duas palavras 

É possível observar similaridade entre duas palavras usando a função __similarity__ do modelo. O retorno é um coeficiente que está entre 0 e 1, sendo que quanto mais próximo de 1, mais próximas as palavras estão no espaço vetorial e, portanto, mais similares elas são. Abaixo, alguns exemplos foram dispostos:

In [71]:
model.wv.similarity(w1="bolsonaro",w2="pt")

0.055565123

In [73]:
model.wv.similarity(w1="dilma",w2="presidente")

0.23505023

In [80]:
model.wv.similarity(w1="silas",w2="gay")

0.54128206

Vê-se portanto que os termos: __bolsonaro__ e __pt__ estão pouco relacionados; __dilma__ e __presidente__ possuem uma proximidade mais significativa; __silas__ e __gay__ tem ainda mais proximidade.

### Palavra estranha

A função __doesnt_match__ por outro lado é útil na busca por palavras que menos se encaixam com o resto em um conjunto de palavras. Num conjunto de palavras em que se tem: __bolsonaro__, __pt__ e __dilma__, por exemplo, espera-se que bolsonaro seja a palavra menos relacionada.

In [61]:
model.wv.doesnt_match(["bolsonaro", "pt", "dilma"])

'bolsonaro'

In [62]:
model.wv.doesnt_match(["cachorro", "ingles", "frances"])

'cachorro'

In [68]:
model.wv.doesnt_match(["pastor", "brasil", "silas"])

'brasil'

### Top 10 similares/mais próximas

Por último, e principal objetivo desta atividade, é possível encontrar os __n termos mais próximos__ de uma palavra, usando a função __most_similar__. Nesta atividade, foi escolhido descobrir os dez termos mais parecidos com: __ciro__; __pt__; __bolsonaro__. Abaixo é possível ver os resultados obtidos:

In [82]:
model.wv.most_similar (positive="ciro", topn=10)

[('nogueira', 0.6924452781677246),
 ('gomes', 0.6457257866859436),
 ('cid', 0.6340566277503967),
 ('pppi', 0.6028022766113281),
 ('pi', 0.5905863642692566),
 ('pros', 0.5849493741989136),
 ('camilo', 0.5513368844985962),
 ('eunicio', 0.4734727144241333),
 ('ceara', 0.46999669075012207),
 ('pp', 0.46808090806007385)]

In [59]:
model.wv.most_similar (positive="pt", topn=10)

[('petista', 0.6563131213188171),
 ('petistas', 0.5065073370933533),
 ('partido', 0.4542264938354492),
 ('dem', 0.38896244764328003),
 ('oposicionista', 0.36980316042900085),
 ('psb', 0.36759695410728455),
 ('pc', 0.36663487553596497),
 ('legenda', 0.3600459694862366),
 ('pr', 0.3457217812538147),
 ('medo', 0.3447813093662262)]

In [60]:
model.wv.most_similar (positive="bolsonaro", topn=10)

[('jair', 0.8037880659103394),
 ('pprj', 0.7005596160888672),
 ('feliciano', 0.679299533367157),
 ('wyllys', 0.6612508296966553),
 ('psolrj', 0.6033853888511658),
 ('jean', 0.5905729532241821),
 ('heinze', 0.571237325668335),
 ('malafaia', 0.5456973314285278),
 ('comprometedor', 0.5443352460861206),
 ('jandira', 0.5385400056838989)]

## Conclusões 

Prova-se nessa atividade o quanto é simples a construção de um modelo __Word2Vec__ usando a biblioteca __gensim__ e, ao mesmo tempo, o quanto os resultados podem ser relavantes. Alguns casos de uso possíveis para este modelo são, por exemplo: entender como estão falando de um produto, num banco de reviews; construir um dicionário de sinônimos; sugerir palavras conforme se digita.